# Image recognition on the CIFAR-10 dataset

we need to recognize images containing objects from the CIFAR dataset. It contains 60k labelled images of 10 different objects. 

![Picture title](https://miro.medium.com/max/505/1*r8S5tF_6naagKOnlIcGXoQ.png)


## Import libraries and define symbolic constants

### Install wandb to keep track of model performance

In [1]:
!pip install --upgrade wandb
!wandb login ff97f4ffa6b4b35ec56fc229fc572b0ba72ac1fb


[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Import and initialize parameters

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import wandb
from wandb.keras import WandbCallback

wandb.init(project="image-classification-cifar10")


IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLUMNS = 32
INPUT_SHAPE = (IMG_ROWS, IMG_COLUMNS, IMG_CHANNELS)

EPOCHS = 40  # this is how many times re-train the model, each time optimizing its weight and biases
BATCH_SIZE = 64 # this is the number of instances we take from the training set before running the optimizer
VERBOSE = 1 # make it loud
N_CLASSES = 10 # 10 classes in the cifar dataset
VALIDATION_SPLIT = 0.2 # 20% of the training data for the validation

#----------------HYPERPARAMETERS-----------------
CONVOLUTION_FILTERS_SIZE = (3,3)
POOL_SIZE = (2,2)
N_HIDDEN = 512 # neurons in hidden dense layer
DROPOUT = 0.25 # portion of dropout values in the network  
DROPOUT_DEEP = 0.5 # portion of dropout values in the network 
ACTIVATION_FUNCTION_HIDDEN = 'relu' # activation function for the hidden layers
ACTIVATION_FUNCTION_FINAL = 'softmax' # activation function for the output layer 
OPTIMIZER = 'RMSprop' # optimizer, this is how we search for the minimum in the loss function
LOSS_FUNCTION = 'categorical_crossentropy' #loss function, this is what is otimized
METRICS = ['accuracy'] #Our metrics, used to make sure we don't overfit. Computed also on the test set 

wandb.config = {
  "batch_size": BATCH_SIZE, 
  "n_hidden": N_HIDDEN,
  'activation_funciton_hidden': ACTIVATION_FUNCTION_HIDDEN,
  'activation_funciton_final': ACTIVATION_FUNCTION_FINAL,
  'optimizer': OPTIMIZER,
  'loss_function': LOSS_FUNCTION,
  'metric': METRICS,
}


2022-11-12 18:32:13.702874: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 18:32:13.819134: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-12 18:32:13.824232: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-12 18:32:13.824249: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

## Load dataset from Keras

In [3]:
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
 
    #normalize 
    mean = np.mean(x_train,axis=(0,1,2,3))
    std = np.std(x_train,axis=(0,1,2,3))
    x_train = (x_train-mean)/(std+1e-7)
    x_test = (x_test-mean)/(std+1e-7)
 
    y_train =  tf.keras.utils.to_categorical(y_train,N_CLASSES)
    y_test =  tf.keras.utils.to_categorical(y_test,N_CLASSES)

    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = load_data()

#Augmet the dataset
datagen = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range =  0.2,
    horizontal_flip = True, 
)

datagen.fit(x_train)

170498071/170498071 [==============================] - 13s 0us/step


## Build the model

- We use typical CNN architecture with convolutions and pooling

- We make the network deeper. This increases complexity, so we need batch normalization and dropouts. 

- As we go deeper, the number of neurons in the network increases. 

- We then flatten the outputs of the convolution and pass them to a dense layer with a softmax activation function

In [4]:
def build_model():
    model = models.Sequential()

#1st block 
    model.add(
        layers.Convolution2D(
            IMG_ROWS,
            CONVOLUTION_FILTERS_SIZE,
            padding='same',
            activation=ACTIVATION_FUNCTION_HIDDEN, 
            input_shape=INPUT_SHAPE
        )
    )
    model.add(layers.BatchNormalization())
    model.add(
        layers.Convolution2D(
            32,
            CONVOLUTION_FILTERS_SIZE,
            padding='same',
            activation=ACTIVATION_FUNCTION_HIDDEN, 
        )
    )
    model.add(layers.BatchNormalization())
    model.add(
        layers.MaxPooling2D(pool_size=POOL_SIZE)
    )
    model.add(
        layers.Dropout(DROPOUT)
    )

    #2nd layer
    model.add(
        layers.Convolution2D(
            64,
            CONVOLUTION_FILTERS_SIZE,
            padding='same',
            activation=ACTIVATION_FUNCTION_HIDDEN, 
        )
    )
    model.add(layers.BatchNormalization())
    model.add(
        layers.Convolution2D(
            64,
            CONVOLUTION_FILTERS_SIZE,
            padding='same',
            activation=ACTIVATION_FUNCTION_HIDDEN, 
        )
    )
    model.add(layers.BatchNormalization())
    model.add(
        layers.MaxPooling2D(pool_size=POOL_SIZE)
    )
    model.add(
        layers.Dropout(0.3)
    )

    #3rd layer
    model.add(
        layers.Convolution2D(
            128,
            CONVOLUTION_FILTERS_SIZE,
            padding='same',
            activation=ACTIVATION_FUNCTION_HIDDEN, 
        )
    )
    model.add(layers.BatchNormalization())
    model.add(
        layers.Convolution2D(
            128,
            CONVOLUTION_FILTERS_SIZE,
            padding='same',
            activation=ACTIVATION_FUNCTION_HIDDEN, 
        )
    )
    model.add(layers.BatchNormalization())
    model.add(
        layers.MaxPooling2D(pool_size=POOL_SIZE)
    )
    model.add(
        layers.Dropout(0.4)
    )

    #flatten and feed to a softmax layer with N_classes neurons
    model.add(
        layers.Flatten()
    )   
    model.add(
        layers.Dense(
            N_CLASSES,
            activation = ACTIVATION_FUNCTION_FINAL
        ) 
    )

    return model
    
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                        

## Compile the model

- We use RMSprop as optimizer

- The loss function is categorical cross-entropy, this is particularly well-suited for multi-class problems with a one-hot encoding 

- We use accuracy to evaluate the performance of the model

In [5]:
model.compile(
    optimizer=OPTIMIZER,
    loss=LOSS_FUNCTION,
    metrics=METRICS
)

## Train the model

We are now ready to train the model. We need to define the number of epochs and the batch size. 

- Epochs are the number of times the model is exposed to the training dataset. Each time, it will run the optimizer (SGD) and try to minimize the loss function. 

- Batch_size is the number of instances that the optimizer observes before tuning the weights and biases. There are many batches per epoch.

- We split the training data in an 80% training and 20% validation per epoch. The validation set is used to compute the metric and tune hyperparameters, to avoid overfitting.

- We add early stopping, on the loss function on the validation set, with a patience of N epoch. This will stop the optimization if the loss function does not go down for N  consecutive epochs. 

In [9]:
score = model.fit_generator(
    datagen.flow(
        x_train,
        y_train,
        batch_size=BATCH_SIZE
    ),
    epochs=EPOCHS,
    validation_data=(x_test,y_test),
    callbacks=[WandbCallback()]
    )

Epoch 1/40
/tmp/ipykernel_263/4154025429.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  score = model.fit_generator(
782/782 [==============================] - ETA: 0s - loss: 2.1403 - accuracy: 0.3494WARNING:absl:Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 6). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: /work/wandb/run-20221112_183227-18cp0wz1/files/model-best/assets
INFO:tensorflow:Assets written to: /work/wandb/run-20221112_183227-18cp0wz1/files/model-best/assets
wandb: Adding directory to artifact (/work/wandb/run-20221112_183227-18cp0wz1/files/model-best)... Done. 0.0s
782/782 [==============================] - 243s 310ms/step - loss: 2.1403 - accuracy: 0.3494 - val_l

## Test the model on unseen data

In [10]:
test_loss, test_accuracy = model.evaluate(
    x_test,
    y_test,
    batch_size=BATCH_SIZE,
    verbose=VERBOSE
    )
    
#track test results on wandb
wandb.log({
    "test_loss": test_loss, 
    "test_accuracy": test_accuracy
})

157/157 [==============================] - 9s 58ms/step - loss: 0.5499 - accuracy: 0.8290


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ba4822a4-198a-4cdb-8280-0ca8d044b999' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>